In [23]:
import os
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.vectorstores.pinecone import Pinecone
#from domino_data.vectordb import DominoPineconeConfiguration
from mlflow.deployments import get_deploy_client
from langchain_community.embeddings import MlflowEmbeddings
from langchain_community.chat_models import ChatMlflow

import pinecone
from getpass import getpass
import warnings
warnings.filterwarnings('ignore')

In [24]:
#os.environ['ANTHROPIC_API_KEY'] = getpass("Enter Anthropic key:")
#os.environ['OPENAI_API_KEY'] = getpass("Enter OpenAI API key:")
#os.environ['PINECONE_API_KEY'] = getpass("Enter Pinecone API key:")
#os.environ['PINECONE_ENV'] = getpass("Enter Pinecone Environment:")

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY') 
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')
PINECONE_ENV = os.getenv('PINECONE_API_ENV')
os.environ['TOKENIZERS_PARALLELISM'] = 'false'

In [25]:
# Create embeddings to embed queries
embed = MlflowEmbeddings(
    target_uri=os.environ["DOMINO_MLFLOW_DEPLOYMENTS"],
    endpoint="embedding-ada-002ja",
)

In [26]:
text_field = "symptoms"

datasource_name = "MedRAG-JA"

#Vector Access 
conf = DominoPineconeConfiguration(datasource=datasource_name)

api_key = os.environ.get("DOMINO_VECTOR_DB_METADATA", datasource_name)

# initialize pinecone
pinecone.init(
    api_key=api_key,
    environment=PINECONE_ENV,
    openapi_config=conf
)

index_name = "medical-qa-search"
index = pinecone.Index(index_name)

# switch back to normal index for langchain
vectorstore = Pinecone(
    index, embed.embed_query, text_field
)

In [27]:
prompt_template = """Use the following pieces of context to answer the question enclosed within  3 backticks at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.
Please provide an answer which is factually correct and based on the information retrieved from the vector store.
Please also mention any quotes supporting the answer if any present in the context supplied within two double quotes "" .

{context}

QUESTION:```{question}```
ANSWER:
"""
PROMPT = PromptTemplate(template=prompt_template, input_variables=["context","question"])
#
chain_type_kwargs = {"prompt": PROMPT}

In [33]:
rag_llm = ChatOpenAI(
    openai_api_key=OPENAI_API_KEY,
    model_name='gpt-4',
    temperature=0.0
)

In [32]:
# Using the AI Gateway Endpoint
rag_llm = ChatMlflow(
        target_uri=os.environ["DOMINO_MLFLOW_DEPLOYMENTS"],
        endpoint="chat-gpt4-ja"
    )

In [29]:
qa_chain = RetrievalQA.from_chain_type(llm=rag_llm,
                                       chain_type="stuff",
                                       chain_type_kwargs={"prompt": PROMPT},
                                       retriever=vectorstore.as_retriever(search_kwargs={"k": 5}),
                                       return_source_documents=True
                                      )

In [30]:
user_question = input("Please provide the symptoms here :")
result = qa_chain(user_question)

Please provide the symptoms here : What are the symptoms of alcoholic hepatitis?


In [31]:
result['result']

'The symptoms of alcoholic hepatitis include yellowing of the skin and whites of the eyes (jaundice), loss of appetite, nausea and vomiting, abdominal tenderness, fever often low grade, fatigue and weakness, fluid accumulation in the abdomen (ascites), confusion and behavior changes due to a buildup of toxins normally broken down and eliminated by the liver, and kidney and liver failure. Malnutrition is also common in people with alcoholic hepatitis.'

In [34]:
result['source_documents'][0].page_content

'The most common sign of alcoholic hepatitis is yellowing of the skin and whites of the eyes (jaundice). Other signs and symptoms include: Malnutrition is common in people with alcoholic hepatitis. Drinking large amounts of alcohol suppresses the appetite and heavy drinkers get most of their calories from alcohol. Additional signs and symptoms that occur with severe alcoholic hepatitis include: Alcoholic hepatitis is a serious often deadly disease. See your doctor if you:        Loss of appetite Nausea and vomiting Abdominal tenderness Fever often low grade Fatigue and weakness Fluid accumulation in your abdomen (ascites) Confusion and behavior changes due to a buildup of toxins normally broken down and eliminated by the liver Kidney and liver failure Have signs or symptoms of alcoholic hepatitis Cant control your drinking Would like help cutting back on your drinking'